In [0]:
try:
    df = spark.table("final_exam_cluster.default.clean_healthcare")
except:
    df = spark.table("clean_healthcare")

df.show(5)
df.printSchema()

+----------+---+------+--------------------+-------------+----------------+-----------+--------------+
|Patient_ID|Age|Gender|            Symptoms|Symptom_count|         Disease|  age_group|severity_level|
+----------+---+------+--------------------+-------------+----------------+-----------+--------------+
|         1| 29|  Male|fever, back pain,...|            3|         Allergy|      Adult|      Moderate|
|         2| 76|Female|insomnia, back pa...|            3|Thyroid Disorder|     Senior|      Moderate|
|         3| 78|  Male|sore throat, vomi...|            3|       Influenza|     Senior|      Moderate|
|         4| 58| Other|blurred vision, d...|            4|          Stroke|Middle-aged|      Moderate|
|         5| 55|Female|swelling, appetit...|            3|   Heart Disease|Middle-aged|      Moderate|
+----------+---+------+--------------------+-------------+----------------+-----------+--------------+
only showing top 5 rows
root
 |-- Patient_ID: long (nullable = true)
 |--

In [0]:
## Basic stats
df.describe(["Age", "Symptom_count"]).show()

+-------+------------------+-----------------+
|summary|               Age|    Symptom_count|
+-------+------------------+-----------------+
|  count|             25000|            25000|
|   mean|          45.30224|          5.00144|
| stddev|26.106224399182953|1.415795926510675|
|    min|                 1|                3|
|    max|                90|                7|
+-------+------------------+-----------------+



In [0]:
## Number of patients per Disease
from pyspark.sql.functions import count

df_patients_by_disease = (
    df.groupBy("Disease")
      .agg(count("*").alias("patient_count"))
      .orderBy("patient_count", ascending=False)
)

df_patients_by_disease.show()

+----------------+-------------+
|         Disease|patient_count|
+----------------+-------------+
|         Anxiety|          911|
|       Arthritis|          896|
|  Food Poisoning|          871|
|      Depression|          859|
|         Allergy|          858|
|      Bronchitis|          856|
|      Dermatitis|          856|
|Thyroid Disorder|          855|
|        Migraine|          854|
|        Diabetes|          850|
|        COVID-19|          839|
|           Ulcer|          833|
|    Hypertension|          833|
|        Epilepsy|          832|
|             IBS|          830|
|   Liver Disease|          830|
|       Pneumonia|          830|
|     Parkinson's|          826|
|       Influenza|          824|
|        Dementia|          823|
+----------------+-------------+
only showing top 20 rows


In [0]:
## Average Symptom_count per Disease
from pyspark.sql.functions import avg

df_avg_symptoms_by_disease = (
    df.groupBy("Disease")
      .agg(avg("Symptom_count").alias("avg_symptom_count"))
      .orderBy("avg_symptom_count", ascending=False)
)

df_avg_symptoms_by_disease.show()

+--------------------+-----------------+
|             Disease|avg_symptom_count|
+--------------------+-----------------+
|              Anemia|5.169533169533169|
|         Common Cold|5.145341614906832|
|           Pneumonia|5.062650602409638|
|            Diabetes|             5.06|
|           Influenza|5.049757281553398|
|Chronic Kidney Di...|5.039653035935564|
|              Stroke|5.039240506329114|
|             Obesity|5.037851037851038|
|           Gastritis|5.024875621890548|
|             Allergy|5.024475524475524|
|       Liver Disease|5.019277108433735|
|          Bronchitis|5.016355140186916|
|    Thyroid Disorder|5.010526315789473|
|        Hypertension|5.007202881152461|
|           Sinusitis|  5.0062893081761|
|               Ulcer|4.998799519807923|
|         Parkinson's| 4.99636803874092|
|           Arthritis|4.995535714285714|
|             Anxiety|4.992316136114161|
|              Asthma|4.984654731457801|
+--------------------+-----------------+
only showing top

In [0]:
## Patients by age_group and severity_level
df_agegroup_severity = (
    df.groupBy("age_group", "severity_level")
      .count()
      .orderBy("age_group", "severity_level")
)

df_agegroup_severity.show()

+-----------+--------------+-----+
|  age_group|severity_level|count|
+-----------+--------------+-----+
|      Adult|      Moderate| 3550|
|      Adult|        Severe| 2489|
|      Child|      Moderate| 2911|
|      Child|        Severe| 1922|
|Middle-aged|      Moderate| 4185|
|Middle-aged|        Severe| 2755|
|     Senior|      Moderate| 4323|
|     Senior|        Severe| 2865|
+-----------+--------------+-----+



In [0]:
%sql
SELECT
  Disease,
  COUNT(*) AS patient_count
FROM final_exam_cluster.default.clean_healthcare
GROUP BY Disease
ORDER BY patient_count DESC

Disease,patient_count
Anxiety,911
Arthritis,896
Food Poisoning,871
Depression,859
Allergy,858
Bronchitis,856
Dermatitis,856
Thyroid Disorder,855
Migraine,854
Diabetes,850


In [0]:
%sql
SELECT
  age_group,
  AVG(Symptom_count) AS avg_symptom_count
FROM final_exam_cluster.default.clean_healthcare
GROUP BY age_group
ORDER BY avg_symptom_count DESC

age_group,avg_symptom_count
Adult,5.0228514654744165
Senior,5.000417362270451
Child,4.995654872749845
Middle-aged,4.9878962536023055
